<a href="https://colab.research.google.com/github/Asmaasa3d/Pyspark/blob/main/Asmaa_saeed_Spark_and_Python_for_Big_Data_Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 51.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=cac38f954048e79daa93f204f23bd89141a67f2a4a98c152a74819cb30e22538
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Final_Exam').getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [ ]:
df=spark.read.csv("/content/SMSSpamCollection.csv",inferSchema=True,sep='\t')

In [ ]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



### Print the schema

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [ ]:
df2=df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [ ]:
df2.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [ ]:
df2.show(10,truncate=True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
df2.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
text_len=udf(lambda x :len(x),IntegerType())


In [ ]:
df3=df2.withColumn('length',text_len('text'))

### Show the new dataframe

In [ ]:
df3.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [ ]:
import pyspark.sql.functions as fn

In [ ]:
df3.groupBy('class').agg(fn.avg('length').alias('Avg. Lenght')).show()

+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF


In [ ]:
tokenizer = Tokenizer(outputCol="words")

tokenizer.setInputCol("text")



Tokenizer_cdefa594d569

In [ ]:
tokenizer.transform(df3).show()

+-----+--------------------+------+--------------------+
|class|                text|length|               words|
+-----+--------------------+------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|
| spam|FreeMsg Hey there...|   147|[freemsg, hey, th...|
|  ham|Even my brother i...|    77|[even, my, brothe...|
|  ham|As per your reque...|   160|[as, per, your, r...|
| spam|WINNER!! As a val...|   157|[winner!!, as, a,...|
| spam|Had your mobile 1...|   154|[had, your, mobil...|
|  ham|I'm gonna be home...|   109|[i'm, gonna, be, ...|
| spam|SIX chances to wi...|   136|[six, chances, to...|
| spam|URGENT! You have ...|   155|[urgent!, you, ha...|
|  ham|I've been searchi...|   196|[i've, been, sear...|
|  ham|I HAVE A DATE ON ...|   

In [ ]:
df4=tokenizer.transform(df3)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
remover = StopWordsRemover(stopWords=stopwords.words('english'),inputCol='words',outputCol='ham_words')


In [ ]:
remover.transform(df4).show()

+-----+--------------------+------+--------------------+--------------------+
|class|                text|length|               words|           ham_words|
+-----+--------------------+------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...|
| spam|FreeMsg Hey there...|   147|[freemsg, hey, th...|[freemsg, hey, da...|
|  ham|Even my brother i...|    77|[even, my, brothe...|[even, brother, l...|
|  ham|As per your reque...|   160|[as, per, your, r...|[per, request, 'm...|
| spam|WINNER!! As a val...|   157|[winner!!, as, a,...|[winner!!, valued...|
| spam|Had your mobile 1...|   154|[had, your, mobil...|[mobile,

In [ ]:
df5=remover.transform(df4)

In [ ]:
cv = CountVectorizer(inputCol='ham_words',outputCol='vectors')

model = cv.fit(df5)

model.transform(df5).show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                                                           

In [ ]:
df6=model.transform(df5)

In [ ]:
idf = IDF(inputCol="vectors", outputCol="features")
idfModel = idf.fit(df6)
df7 = idfModel.transform(df6)
df7.select("class", "features").show()

+-----+--------------------+
|class|            features|
+-----+--------------------+
|  ham|(13437,[8,12,33,6...|
|  ham|(13437,[0,26,303,...|
| spam|(13437,[2,14,20,3...|
|  ham|(13437,[0,70,82,1...|
|  ham|(13437,[38,137,31...|
| spam|(13437,[11,60,145...|
|  ham|(13437,[11,54,104...|
|  ham|(13437,[130,189,4...|
| spam|(13437,[1,49,124,...|
| spam|(13437,[0,1,14,29...|
|  ham|(13437,[5,19,45,1...|
| spam|(13437,[9,18,39,9...|
| spam|(13437,[14,32,49,...|
|  ham|(13437,[40,97,101...|
|  ham|(13437,[559,1722,...|
| spam|(13437,[32,110,11...|
|  ham|(13437,[83,216,37...|
|  ham|(13437,[0,2,51,13...|
|  ham|(13437,[0,75,107,...|
| spam|(13437,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
df7.select("class", "features").show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|features                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
+-----+-----------------------

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler


In [ ]:
stringIndexer = StringIndexer(inputCol="class", outputCol="indexed_class",handleInvalid='skip',stringOrderType="frequencyDesc")
stringIndexer_model = stringIndexer.fit(df7)

In [ ]:
stringIndexer_model.transform(df7).show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-------------+
|class|                text|length|               words|           ham_words|             vectors|            features|indexed_class|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|(13437,[8,12,33,6...|(13437,[8,12,33,6...|          0.0|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|(13437,[0,26,303,...|(13437,[0,26,303,...|          0.0|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|(13437,[2,14,20,3...|(13437,[2,14,20,3...|          1.0|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|(13437,[0,70,82,1...|(13437,[0,70,82,1...|          0.0|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, 

In [ ]:
df8=stringIndexer_model.transform(df7)

In [ ]:
vec_assem=VectorAssembler(inputCols=['features','length'],outputCol='feature')

In [ ]:
vec_assem.transform(df8).select('feature').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
+-----------------

In [ ]:
train_df=vec_assem.transform(df8)

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [ ]:
from pyspark.ml import Pipeline


In [ ]:
##models
tokenizer = Tokenizer(inputCol='text',outputCol="words")
remover = StopWordsRemover(stopWords=stopwords.words('english'),inputCol='words',outputCol='ham_words')
cv = CountVectorizer(inputCol='ham_words',outputCol='vectors')
idf = IDF(inputCol="vectors", outputCol="features")

stringIndexer = StringIndexer(inputCol="class", outputCol="indexed_class",handleInvalid='skip',stringOrderType="frequencyDesc")

vec_assem=VectorAssembler(inputCols=['features','length'],outputCol='feature')

nb = NaiveBayes(featuresCol='feature',labelCol='indexed_class')

In [ ]:
pipline=Pipeline(stages=[tokenizer,remover,cv,idf,stringIndexer,vec_assem,nb])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [ ]:
train_df,test_df=df3.randomSplit([.7,.3],seed=42)


### Fit your Pipeline model to the training data

In [ ]:
model= pipline.fit(df3)

### Perform predictions on tests dataframe

In [ ]:
predictions = model.transform(test_df)

### Print the schema of the prediction dataframe

In [ ]:
predictions.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ham_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- indexed_class: double (nullable = false)
 |-- feature: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
score = MulticlassClassificationEvaluator().setLabelCol("indexed_class").setPredictionCol("prediction").setMetricName("fMeasureByLabel")

In [ ]:

score.evaluate(predictions)

0.9963689179375453

# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>